<a href="https://colab.research.google.com/github/shubhamgupta0903/Potato_disease_classification/blob/main/Potato_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive to '/content/gdrive'
drive.mount('/content/gdrive')






Mounted at /content/gdrive


In [ ]:
import os
from PIL import Image



# Define the path to your dataset
dataset_path = '/content/gdrive/My Drive/PlantVillage'

# List all folders in the dataset
folders = os.listdir(dataset_path)

for folder in folders:
    folder_path = os.path.join(dataset_path, folder)


    files = os.listdir(folder_path)


    for file in files:
        file_path = os.path.join(folder_path, file)






In [ ]:
IMAGE_SIZE=256
BATCH_SIZE=32
CHANNELS=3
EPOCHS=50

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

Found 2282 files belonging to 3 classes.


In [ ]:
class_names=dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [ ]:
len(dataset)

72

In [ ]:
train_size=0.8
len(dataset)*train_size

57.6

In [ ]:
train_ds=dataset.take(57)

In [ ]:
test_ds=dataset.skip(57)

In [ ]:
val_size=0.1
len(dataset)*val_size

7.2

In [ ]:
val_ds=test_ds.take(7)

In [ ]:
test_ds=test_ds.skip(7)

In [ ]:
def get_dataset_partition_tf(ds,train_split=0.8,test_split=0.1,val_split=0.1,shuffle=True,shuffle_size=1000):
  ds_size=len(ds)
  if shuffle:
    ds=ds.shuffle(shuffle_size,seed=12)
  train_size=int(train_split*ds_size)
  val_size=int(val_split*ds_size)

  train_ds=ds.take(train_size)

  val_ds=ds.skip(train_size).take(val_size)
  test_ds=ds.skip(train_size).skip(val_size)

  return train_ds,val_ds,test_ds

In [ ]:
train_ds,val_ds,test_ds=get_dataset_partition_tf(dataset)

In [ ]:
len(train_ds
    )

57

In [ ]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_and_rescale=tf.keras.Sequential([
    keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    keras.layers.experimental.preprocessing.Rescaling(1.0/255)
]

)

In [ ]:
data_augmentation=tf.keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
from tensorflow.keras.models import Sequential
n_classes=3
model=Sequential([
    resize_and_rescale,
    data_augmentation,
    keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(n_classes,activation='softmax'),

])

In [ ]:
model.build(input_shape=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 sequential_1 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (32, 127, 127, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (32, 62, 62, 64)          0         
 g2D)                                                 

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']

)

In [ ]:
model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,


    validation_data=val_ds
)

Epoch 1/50
57/57 [==============================] - 744s 180ms/step - loss: 0.8818 - accuracy: 0.4951 - val_loss: 0.7648 - val_accuracy: 0.6339
Epoch 2/50
57/57 [==============================] - 3s 61ms/step - loss: 0.5851 - accuracy: 0.7220 - val_loss: 0.5205 - val_accuracy: 0.7500
Epoch 3/50
57/57 [==============================] - 3s 60ms/step - loss: 0.3489 - accuracy: 0.8629 - val_loss: 0.2208 - val_accuracy: 0.9330
Epoch 4/50
57/57 [==============================] - 3s 60ms/step - loss: 0.2972 - accuracy: 0.8854 - val_loss: 0.3547 - val_accuracy: 0.9062
Epoch 5/50
57/57 [==============================] - 3s 61ms/step - loss: 0.1386 - accuracy: 0.9507 - val_loss: 0.0538 - val_accuracy: 0.9732
Epoch 6/50
57/57 [==============================] - 3s 59ms/step - loss: 0.1625 - accuracy: 0.9353 - val_loss: 0.1355 - val_accuracy: 0.9464
Epoch 7/50
57/57 [==============================] - 3s 59ms/step - loss: 0.1179 - accuracy: 0.9534 - val_loss: 0.0530 - val_accuracy: 0.9821
Epoch 8/50

In [ ]:
score=model.evaluate(test_ds)

8/8 [==============================] - 6s 27ms/step - loss: 0.1409 - accuracy: 0.9648


In [ ]:
history

from google.colab import drive

# Mount Google Drive to '/content/gdrive'
drive.mount('/content/gdrive')
import os
from PIL import Image  # Assuming you want to work with images



# Define the path to your dataset
dataset_path = '/content/gdrive/My Drive/PlantVillage'

# List all folders in the dataset
folders = os.listdir(dataset_path)

# Iterate through each folder
for folder in folders:
    folder_path = os.path.join(dataset_path, folder)

    # List all files (images) in the folder
    files = os.listdir(folder_path)

    # Iterate through each file (image) in the folder
    for file in files:
        file_path = os.path.join(folder_path, file)

        # Do something with the file, for example, display the image
IMAGE_SIZE=256
BATCH_SIZE=32
CHANNELS=3
EPOCHS=50
import tensorflow as tf
from tensorflow import keras
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)
cl

In [ ]:
model.save('/content/drive/My Drive/potato_disease.ipynb')


In [ ]:
model_version=1
model.save(f"../models/{model_version}")

In [ ]:
import os

# List the contents of the directory
print(os.listdir('/content/../models'))


['1']


In [ ]:
print(os.path.abspath(f"../models/{model_version}"))


/models/1
